In [42]:
"""
 - create binary tree
 - make laplacian
 - solve = Tutte
 - eigenvalues = Spectral
 - gradient descent for eades
 - ?sgd for stress

        0
      /   \
    1       2
   / \     / \
  3   4   5   6

children = x*2+1, x*2+2
parent = (x-1)/2
"""

import numpy as np

def maketree(depth):
    I = []
    J = []
    def dfs(idx, depth):
        if depth <= 0:
            return
        parent = (idx-1)//2
        I.append(parent)
        J.append(idx)
        I.append(idx)
        J.append(parent)

        dfs(idx*2+1, depth-1)
        dfs(idx*2+2, depth-1)

    # node 0 is root and so has no parent
    dfs(1, depth-1)
    dfs(2, depth-1)
    return I,J

In [65]:
depth = 4
I,J = maketree(depth)
n = (1<<depth)-1
A = np.zeros((n,n))
for i,j in zip(I,J):
    A[i,j] = 1

firstleaf = (1<<(depth-1))-1
print(f'{depth} {n} {firstleaf}')

4 15 7


In [66]:
import math
import numpy.linalg as alg

# calculate Laplacian matrix
L = np.zeros((n,n))
for i,j in zip(I,J):
    L[i,j] -= 1
    L[i,i] += .5
    L[j,j] += .5

In [ ]:
# Tutte: fix leaf vertices then solve

b = np.zeros((n,2))
nleaves = (n-firstleaf)
for i in range(firstleaf, n):
    L[i,:] = 0
    L[i,i] = 1
    rad = (i-firstleaf)/nleaves * 2*math.pi
    b[i,0] = math.cos(rad)
    b[i,1] = math.sin(rad)
    
X = alg.solve(L, b)

In [72]:
# Spectral: calculate eigenvalues

w,v = alg.eig(L)

idx = w.argsort()[::-1]

# sort by eigenvalue
w = w[idx]
v = v[:,idx]

# take second and third eigenvectors
X = v[:,1:3]

In [73]:
import s_gd2

s_gd2.draw_png(X, I, J, 'tree.png', linkwidth=.02, noderadius=.025)